# Estimating the average X-ray backgound at high redshifts

At very high redshifts, the primary source of X-rays will be from High-Mass X-ray Binaries (HMXBs).  Given an estimate of the Star Formation Rate Density (SFRD), we can calculate the corresponding energy density of X-rays in space ($u_{\rm \small XR}$) by assuming values for the typical number, mass, luminosity and lifespan of HMXBs:

$$u_{\rm \small XR}(z) = f_{\rm \small HMXB} \Psi_{*}(z) \langle\frac{L_{\rm \small XR}}{M_{\rm \small HMXB}}\rangle \Delta t_{\rm \small HMXB} \; t_{\rm \small H}(z) (1+z)^3,$$

where $$f_{\rm \small HMXB} \equiv {\rm fraction \; of \; stars \; by \; mass\; dying \; as \; HMXBs}$$

$$\Psi_*(z) \equiv {\rm comoving \; SFR \; in \;M}_{\odot} \; {\rm yr}^{-1} \; {\rm Mpc}^{-3}$$

$$L_{\rm \small XR} \equiv {\rm HMXB \; X\!-\!ray \; luminosity}$$

$$M_{\rm \small HMXB} \equiv {\rm HMXB \; mass}$$

$$\Delta t_{\rm \small HMXB} \equiv {\rm typical \; HMXB \; lifespan}$$

$$t_{\rm \small H} \equiv {\rm Hubble \; time \; at \; redshift \;}z$$

Here, $t_{\rm \small H}$ accounts for the time X-rays have had to propagate through the universe since their creation, and the factor of $(1+z)^3$ is required to convert from a comoving SFRD to a physical X-ray energy density.

## HMXB (High-Mass X-Ray Binary) property assumptions:

The fraction of stars by mass that end up as HMXBs is determined by the mass fraction of stars forming black holes, the fraction of black holes in binary systems, and the fraction of binaries close enough for mass transfer to
occur. For Population III (Pop III) stars, we estimate this efficiency factor as $$ f_{\rm \small HMXB} = f_{\rm \small BH} f_{\rm binary} f_{\rm close} \simeq \left(\frac{1}{2}\right) \left(\frac{1}{2}\right) \left(\frac{2}{15}\right) = \frac{1}{30}$$

In [ ]:
f_xrb = 1./30.

We assume the typical lifetime of a HMXB is $10^7$ years.

In [ ]:
t_xrb = 1e7 

HMXBs will accrete material at close to the Eddington limit,
so we can approximate

$$\frac{L_{\rm \small HMXB}}{M_{\rm \small HMXB}} \simeq l_{\rm \small EDD} = 10^{38} {\rm \; erg\;s}^{-1} {\rm \; M_{\odot}}^{-1}$$

But, this gives us an estimate of the total bolometric luminosity of the HMXB, not the X-Ray luminosity.  For a $\rm 100 \; M_{\odot}$ BH accreting at a rate of $\rm 10^{-6} \; M_{\odot} \; yr^{-1}$, approximately 30% of the total luminosity is between 1 keV and 10 keV.  So, 
$$L_{\rm \small XR} = \frac{1}{3} L_{\rm \small HMXB}$$

In [ ]:
l_xrb = 1e38 / 3

## Assumed Star Formation Rate Density:

We use the star formation rate density determined by Greif & Bromm (2006).  To do so we load the file 'sfr_GB06.dat' containing the comoving SFR as a function of redshift in units of solar masses per year per cubic Mpc. This was extracted from Figure 7 (dotted line) of GB06 and extended to redshift 100 using a log-linear extrapolation.

In [ ]:
import os
from astropy.io import ascii
fname = '../jxr/sfr_GB06.dat'
sfdata = ascii.read(fname)
z = sfdata['z']
sfr = sfdata['sfr']

In [ ]:
def find_closest(A, target):
    #A must be sorted
    idx = A.searchsorted(target)
    idx = np.clip(idx, 1, len(A)-1)
    left = A[idx-1]
    right = A[idx]
    idx -= target - left < right - target
    return idx

## Calculate $u_{\rm \small XR}$:

First calculate the hubble time for each redshift point and convert it to seconds.

In [ ]:
import astropy
from astropy.cosmology import WMAP9
from astropy import units
t_H = WMAP9.age(z) #returns age of universe in Gyr
t_s = t_H.to('s').value #just want the array, not the unit information

z3 = (1+z)**3 # (1+z)^3 (defined for convenience)

Now calculate $u_{\rm \small XR}$ and convert to cgs units.

In [ ]:
# X-ray energy density (erg/Mpc^3)
u_xr = f_xrb * t_xrb * l_xrb * t_s * sfr * z3
# Convert to cgs units (Mpc --> cm)
u_xr /= (3.08e24)**3

## Calculate $J_{\rm \small XR}$:

The average intensity of this radiation, $J_{\rm \small XR}(z)$ is then given by
$$J_{\rm \small XR}(z) = \int_{1 {\rm keV}}^{10 {\rm keV}} J_{\rm \small XR}(z,\nu) d\nu = \frac{c}{4\pi} u_{\rm \small XR}(z)$$

In [ ]:
import numpy as np
J_xr = u_xr * 2.99792458e+10 / (4*np.pi)

Following Inayoshi & Omukai (2011), we define
$$J_{\rm \small XR}(z,\nu) = J_0(z) \left(\frac{\nu}{\nu_0}\right)^{-1.5} d\nu$$

In [ ]:
h = 4.135667516e-15
from scipy import integrate
def J_nu(v):
    return (v/(1e3/h))**-1.5

This allows us to calculate the normalization factor $J_{\rm \small 0}(z)$ for plugging into our simulations.

In [ ]:
integral = integrate.quad(J_nu, 1e3/h, 1e4/h)
J0 = J_xr / integral[0] # integrate.quad returns tuple with result + abserror

### Save this to the file 'jxr.dat' for loading into GADGET.

In [ ]:
jdata = np.zeros((z.size,2))
jdata[:,0] = z[::-1]
jdata[:,1] = J0[::-1]
np.savetxt('jxr.dat', jdata, fmt='%.8e %.8e')

# Now let's make some plots.

In [ ]:
%matplotlib inline
from matplotlib import pyplot as plt
import seaborn as sns
sns.set_style('white')
sns.set_context('poster')

First lets plot the SFR as a function of redshift:

In [ ]:
plt.rcParams['figure.figsize'] = (8,6)
plt.rcParams['text.usetex'] = True
plt.rc('font', size=18.)
plt.semilogy(z,sfr, 'k')
plt.xlabel('Redshift')
plt.ylabel(r'SFR [M$_{\odot}$ yr$^{-1} Mpc$^{-3}$]')
plt.savefig('sfr.png')

Now plot u<sub>XR</sub> as a function of redshift:

In [ ]:
plt.plot(z,np.log10(u_xr), 'k')
plt.xlabel('Redshift')
plt.ylabel(r'Log u$_{\textsc{xr}}$ [erg cm$^{-3}$]')
plt.savefig('u_xr.png', bbox_inches='tight')

And then, our various models for J<sub>XR</sub>:

In [ ]:
sns.palplot(sns.color_palette())

In [ ]:
current_palette = sns.color_palette()
fig, ax = plt.subplots()
ax.axvspan(25.04, 25.84, color='.9', zorder=1)

for i,hm in [(2,1e6),(0,2e5)]:
    #gammaCrit = 2.6e-7 * ((hm/1e6)**(2./3.)) *((1+z)/20)**(2.5)
    gammaCrit = 3.4e-4 * ((hm/1e6)**(2./3.)) *((1+z)/20)**(2.5)
    ax.fill_between(z, np.log10(gammaCrit),np.log10(gammaCrit/2), color=current_palette[i], zorder=2)
    #ax.plot(z, np.log10(gammaCrit), color=current_palette[i])
ax.plot(z, np.log10(J_xr), '.7')
ax.plot(z, np.log10(J_xr*10), '.5')
ax.plot(z, np.log10(J_xr*100), '.3')
ax.plot(z, np.log10(J_xr*1000), '.1')


ax.set_xlim(20,50)
ax.set_ylim(-8,-2)
ax.set_xlabel('Redshift')
ax.set_ylabel(r'Log J$_{\textsc{xr}}$ [erg s$^{-1}$ cm$^{-2}$ sr$^{-1}$]')
#fig.savefig('figures/J_xr.png', bbox_inches='tight')
fig.savefig('figures/J_xr.pdf', bbox_inches='tight')